In [27]:
import os
import sys
import torch
import torch.nn as nn
from torchvision.models import resnet50, ResNet50_Weights, vit_b_16, ViT_B_16_Weights

sys.path.append("python")
from local_utils import print_parameters

In [39]:
model_architecture = ["resnet50", "swin_t", "vit_b16"][2]
seed = 19
# checkpoint_path = "../model_weights/PDDD/ResNet50-Plant-model-80.pth"
# checkpoint_path = "../model_weights/PDDD/vit_base_patch16_224_in21k-model-197.pth"
# checkpoint_path = "../model_weights/PDDD/swin_tiny_patch4_window7_224-model-97.pth"
# checkpoint_path = None
num_classes = None
# pretrained = True
pretrained = False
# num_classes = 120

# weights_path = f"../model_weights/random/ResNet50-Random_{seed}.pth"
# weights = None

# weights_path = "../model_weights/imagenet/ResNet50-ImageNet_v1.pth"
# weights_path = "../model_weights/imagenet/ResNet50-ImageNet_v2.pth"
# weights = ResNet50_Weights.IMAGENET1K_V1
# weights = ResNet50_Weights.IMAGENET1K_V2
# weights = ViT_B_16_Weights.IMAGENET1K_V1
# weights = ViT_B_16_Weights.IMAGENET1K_SWAG_E2E_V1
weights = ViT_B_16_Weights.IMAGENET1K_SWAG_LINEAR_V1

# weights_path = f"../model_weights/random/Swin_T-Random_{seed}.pth"
# weights_path = "../model_weights/imagenet/Swin_T-ImageNet.pth"
# weights_path = f"../model_weights/random/ViT_B16-Random_{seed}.pth"
# weights_path = "../model_weights/imagenet/ViT_B16-ImageNet_v1.pth"
# weights_path = "../model_weights/imagenet/ViT_B16-ImageNet_v2.pth"
weights_path = "../model_weights/imagenet/ViT_B16-ImageNet_v3.pth"

assert model_architecture in weights_path.lower()

In [40]:
torch.manual_seed(seed)

In [41]:
model = None

if model_architecture == "resnet50":
    model = resnet50(weights=weights)
    print(f"Output size: {model.fc.out_features}")
    if num_classes is not None:
        model.fc = nn.Linear(model.fc.in_features, num_classes)

elif model_architecture == "swin_t":
    print(f"pretrained: {pretrained}")
    model = torch.hub.load(
        "SharanSMenon/swin-transformer-hub:main",
        "swin_tiny_patch4_window7_224",
        pretrained=pretrained,
    )
    print(f"Output size: {model.head.out_features}")
    if num_classes is not None:
        model.head = nn.Linear(model.head.in_features, num_classes)
elif model_architecture == "vit_b16":
    print(f"pretrained: {pretrained}")
    # model = torch.hub.load(
    #     "facebookresearch/dino:main",
    #     "dino_vitb16",
    #     pretrained=pretrained,
    # )
    model = vit_b_16(weights=weights)
else:
    print(f"Unknown model architecture: {model_architecture}")

print_parameters(model)

pretrained: False


Downloading: "https://download.pytorch.org/models/vit_b_16_lc_swag-4e70ced5.pth" to C:\Users\peter/.cache\torch\hub\checkpoints\vit_b_16_lc_swag-4e70ced5.pth
100%|██████████| 330M/330M [00:05<00:00, 61.6MB/s] 


Total parameters: 86567656
Required parameters: 86567656 


(86567656, 86567656)

In [42]:
if checkpoint_path is not None:
    checkpoint = torch.load(checkpoint_path, map_location=torch.device("cpu"))
    model.load_state_dict(checkpoint, strict=False)

In [43]:
print_parameters(model)  # 25557032 / 28288354
if os.path.exists(weights_path):
    print(f"File {weights_path} already exists")
else:
    torch.save(model.state_dict(), weights_path)
    print(f"File {weights_path} saved")

Total parameters: 86567656
Required parameters: 86567656 
File ../model_weights/imagenet/ViT_B16-ImageNet_v3.pth saved
